# Pydantic Basics: Creating and Using Models

Pydantic models are the foundation of data validation in Python. They use python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples.

### Python class

In [20]:
class Person:
    def __init__(self, name, age, city):
        self.name = name
        self.age = age
        self.city = city

    def __repr__(self):
        return f"Person(name='{self.name}', age={self.age}, city='{self.city}')"

    # 🔧 Custom function
    def details(self):
        return f"Name: {self.name}, Age: {self.age}, City: {self.city}"

# Create object
p = Person("Ankita", 30, "Kamloops")

# Print using __repr__
print(p)

# Call custom function
print(p.details())


Person(name='Ankita', age=30, city='Kamloops')
Name: Ankita, Age: 30, City: Kamloops


In [21]:
p = Person("Ankita", 30, 35)
print(p)

Person(name='Ankita', age=30, city='35')


### Data Class

In [22]:
from dataclasses import dataclass

@dataclass
class Person():
    name:str
    age:int
    city:str

person = Person(name="Anamica", age=73, city="Kamloops")
print(person)

Person(name='Anamica', age=73, city='Kamloops')


In [15]:
person = Person(name="Anamica", age=73, city="Kamloops")
print(person)

Person(name='Anamica', age=73, city='Kamloops')


In [ ]:
person = Person(name="Anamica", age=73, city=35) # No validation here for city
print(person)

Person(name='Anamica', age=73, city=35)


No typecheck. No validation. So we are going to use pydantic.


In [23]:
from pydantic import BaseModel

In [24]:
class Person1(BaseModel):
    name: str
    age: int
    city: str

person1 = Person1(name="Anamika", age = 35, city = "Banglore" )
print(person1)
person2 = Person1(name="Anamika", age = 35, city = 35 )
print(person2)

name='Anamika' age=35 city='Banglore'


ValidationError: 1 validation error for Person1
city
  Input should be a valid string [type=string_type, input_value=35, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

# Model with Optional Fields

Add optional fields using Python's Optional type:


In [25]:
from typing import Optional
class  Employee(BaseModel):
    id: int
    name: str
    department: str
    salary: Optional[float]=None #Optional with default value
    is_active: Optional[bool]= True #Optional field with default value

In [26]:
emp1 = Employee(id=1,name="John", department = "CS")
emp1

Employee(id=1, name='John', department='CS', salary=None, is_active=True)

In [27]:
emp2 = Employee(id=1,name="John", department = "CS", salary="30000")
emp2

Employee(id=1, name='John', department='CS', salary=30000.0, is_active=True)

In [31]:
emp3 = Employee(id=1,name="John", department = "CS", salary="30000", is_active=23 )
emp3

ValidationError: 1 validation error for Employee
is_active
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value=23, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/bool_parsing

In [32]:
emp4 = Employee(id=1,name="John", department = "CS", salary="kk", is_active=23 )
emp4

ValidationError: 2 validation errors for Employee
salary
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='kk', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing
is_active
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value=23, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/bool_parsing

Definition:
- Optional[type]: Indicates the field can be None

- Default value (= None or = True): Makes the field optional

- Required fields must still be provided

- Pydantic validates types even for optional fields when values are provided

In [33]:
from typing import List

class Classroom(BaseModel):
    room_number: str
    students: List[str] # List of string
    capacity: int


In [15]:
# Create a classroom

classroom = Classroom(
    room_number="A101",
    students=("Aliice","Bob","Charlie"),
    capacity=30
)
print(classroom)

room_number='A101' students=['Aliice', 'Bob', 'Charlie'] capacity=30


In [16]:
classroom = Classroom(
    room_number="A101",
    students=("Aliice",123,"Charlie"),
    capacity=30
)
print(classroom)

ValidationError: 1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [34]:
try:
    invalid_val=Classroom(room_number="A1",students=["Krish",123],capacity=30)

except Exception as e:
    print(e)

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


# Model with Nested Models
Create complex structures with nested models

In [35]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    zip_code: str

class Customer(BaseModel):
    customer_id: int
    name: str
    address: Address #Nested Model

In [36]:
customer = Customer(customer_id=1, name="Ankita", address={"street": "Main street", "city":"Boston","zip_code":"02103"})
print(customer)    

customer_id=1 name='Ankita' address=Address(street='Main street', city='Boston', zip_code='02103')


In [37]:
customer = Customer(customer_id=1, name="Ankita", address={"street": "Main street", "city":123,"zip_code":"02103"})
print(customer)    

ValidationError: 1 validation error for Customer
address.city
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

#### Pydantic Fields: Customization and Constraints

The Field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's a comprehensive tutorial with examples.

In [41]:
from pydantic import BaseModel, Field

class Item(BaseModel):
    name:str=Field(min_length=2,max_length=50)
    price:float=Field(gt=0, le=10000) # greater than 0 or less than or equal to 10000
    quantity:int=Field(ge=0)

item=Item(name="Book", price=10, quantity=10)
print(item)



name='Book' price=10.0 quantity=10


In [42]:
item=Item(name="Book", price=10000000, quantity=10)
print(item)


ValidationError: 1 validation error for Item
price
  Input should be less than or equal to 10000 [type=less_than_equal, input_value=10000000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

In [44]:
class User(BaseModel):
    username:str=Field(description="Unique username for the user")
    age:int=Field(default=18,description="User age default to 18 ")
    email:str= Field(default_factory=lambda: "user@example.com",description="Default email address")


# Examples
user1 = User(username="alice")
print(user1)


username='alice' age=18 email='user@example.com'


In [45]:
user2 = User(username="bob", age=25, email="bob@domain.com")
print(user2)

username='bob' age=25 email='bob@domain.com'


In [46]:
User.model_json_schema()

{'properties': {'username': {'description': 'Unique username for the user',
   'title': 'Username',
   'type': 'string'},
  'age': {'default': 18,
   'description': 'User age default to 18 ',
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'Default email address',
   'title': 'Email',
   'type': 'string'}},
 'required': ['username'],
 'title': 'User',
 'type': 'object'}

In [ ]:
z